# Task 5D – Teil 5: ML-basierte Gesprächszuordnung (Pair-Klassifikation)

**Umfang dieses Notebooks (Teil 5):**

1. Laden der vorhandenen Embeddings (`.npz`) und Metadaten (`.parquet`) inklusive Cluster-Informationen  
2. Konstruktion eines Paar-Datasets: Bildung von positiven und negativen Sprecherpaaren innerhalb einer Session  
3. Extraktion relevanter Merkmale (u. a. Cosine Similarity, zeitlicher Abstand, Sprecherwechsel, Überlappung, Textlänge)  
4. Training klassischer ML-Modelle (Logistische Regression, Random Forest, optional SVM) zur Klassifikation der Paarzugehörigkeit  
5. Evaluation anhand geeigneter Metriken (Accuracy, Precision, Recall, F1, ROC-AUC, Konfusionsmatrix)  
6. Analyse der Feature-Bedeutung und Fehlklassifikationen zur Interpretation der Ergebnisse  
7. Diskussion der Integration dieses Ansatzes in die bestehende Task-5D-Pipeline und Vergleich mit dem distanzbasierten Clustering aus Teil 4

> **Hinweis:** Dieses Notebook erweitert die Kernkomponente des Task-5D-Prototyps um einen ML-basierten Klassifikationsansatz. Ziel ist es, die Robustheit der Gesprächstrennung zu erhöhen, indem semantische und zeitliche Merkmale kombiniert und direkt zur Vorhersage der Gesprächszugehörigkeit genutzt werden.


## 1. Imports, Reproduzierbarkeit & Setup

In diesem Abschnitt werden die benötigten Bibliotheken geladen, Zufallssamen gesetzt sowie Pandas-Optionen für die Ausgabe definiert.  
Darüber hinaus erfolgt die Basiskonfiguration (Pfadangaben, Sampling-Parameter).  
Standardmäßig werden die Daten per **Auto-Discovery** gesucht, können bei Bedarf aber auch manuell überschrieben werden.


In [1]:
# ------------------------------------------------------------
# 1) Imports, Reproduzierbarkeit & Setup
# ------------------------------------------------------------

# --- System / Utility ---
from pathlib import Path
import json
import math
import warnings

# --- Numerik & Datenverarbeitung ---
import numpy as np
import pandas as pd

# --- Fortschrittsbalken ---
from tqdm.auto import tqdm   # für dynamische Fortschrittsanzeigen im Notebook

# --- Machine Learning (Scikit-Learn) ---
from sklearn.model_selection import GroupShuffleSplit, GroupKFold   # gruppenbasierte Splits nach Session
from sklearn.pipeline import Pipeline                               # Verarbeitungsketten (z. B. Scaling + Modell)
from sklearn.preprocessing import StandardScaler                    # Standardisierung numerischer Features
from sklearn.metrics import (                                       # gängige Klassifikationsmetriken
    accuracy_score, precision_recall_fscore_support,
    roc_auc_score, classification_report, confusion_matrix
)
from sklearn.linear_model import LogisticRegression                 # Logistische Regression
from sklearn.ensemble import RandomForestClassifier                 # Random Forest
from sklearn.svm import SVC                                         # Support Vector Machine (optional)

# --- Visualisierung ---
import matplotlib.pyplot as plt

# ------------------------------------------------------------
# Reproduzierbarkeit & Anzeigeoptionen
# ------------------------------------------------------------

# Zufalls-Seed für Numpy -> sorgt für reproduzierbare Ergebnisse
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Pandas-Optionen für bessere Übersicht im Notebook
pd.set_option("display.max_colwidth", 200)   # volle Anzeige langer Texte
pd.set_option("display.width", 160)          # max. Zeilenbreite
pd.set_option("display.max_columns", 120)    # max. Spaltenanzahl

# Warnungen unterdrücken (z. B. sklearn-Deprecations), um saubere Ausgabe zu erhalten
warnings.filterwarnings("ignore")

# ------------------------------------------------------------
# Verzeichnis-Setup
# ------------------------------------------------------------

def ensure_dir(p: Path):
    """
    Legt ein Verzeichnis (inkl. Unterordnern) an, falls es noch nicht existiert.
    Entspricht dem Verhalten von 'mkdir -p'.
    """
    p.mkdir(parents=True, exist_ok=True)

# Projekt-Root (einheitlich mit Notebook 2–4 setzen!)
PROJECT_ROOT = Path.home() / "AUVIS/task5D_ml_prototype"

# Unterordner für Ergebnisse von Teil 5 (Pair-Dataset)
DERIVED_DIR = PROJECT_ROOT / "data" / "derived" / "part5_pairs"
ensure_dir(DERIVED_DIR)

print(f"PROJECT_ROOT: {PROJECT_ROOT}")


PROJECT_ROOT: /home/ercel001/AUVIS/task5D_ml_prototype


## 2. Auto-Discovery von Dateien

In diesem Schritt werden die benötigten Dateien automatisch gesucht:

- **Embeddings (.npz):** enthalten `utt_ids` und `embeddings`  
- **Metadaten (.parquet):** enthalten u. a. `utt_id`, `speaker_id`, `session_id`, Zeitstempel (`start`, `end`) sowie die Cluster-Zugehörigkeit

> Falls keine passenden Dateien gefunden werden, können die Pfade im nächsten Abschnitt manuell gesetzt werden.


In [2]:
# ============================================================
# 2) Auto-Discovery von Dateien (mit Duplikat-Filter)
# ============================================================

SEARCH_DIRS = [
    PROJECT_ROOT / "data",        # reicht aus – keine Überlappungen nötig
    Path("/mnt/data"),
]

def find_files(extensions=("npz", "parquet", "json")):
    found = {"npz": [], "parquet": [], "json": []}

    for d in SEARCH_DIRS:
        if not d.exists():
            continue
        for ext in extensions:
            for p in d.rglob(f"*.{ext}"):
                if ext == "npz" and "emb" in p.name.lower():
                    found["npz"].append(p)
                elif ext == "parquet" and ("utterances" in p.name.lower()):
                    found["parquet"].append(p)
                elif ext == "json" and "speaker_to_cluster" in p.name.lower():
                    found["json"].append(p)

    # Duplikate entfernen & sortieren
    for k in found:
        found[k] = sorted(set(found[k]))
    return found

found = find_files()

print("Gefundene Embeddings (.npz):")
for p in found["npz"]:
    print("  -", p)

print("\nGefundene Metadaten (.parquet):")
for p in found["parquet"]:
    print("  -", p)

print("\nGefundene speaker_to_cluster.json:")
for p in found["json"]:
    print("  -", p)

def pick_latest(paths):
    if not paths:
        return None
    return max(paths, key=lambda p: p.stat().st_mtime)

# Standard-Dateien (Fallback)
DEFAULT_EMB_NPZ = pick_latest(found["npz"])
DEFAULT_META_PARQUET = pick_latest(found["parquet"])
DEFAULT_STC_JSON = pick_latest(found["json"])


Gefundene Embeddings (.npz):
  - /home/ercel001/AUVIS/task5D_ml_prototype/data/embeddings/dev_embeddings.npz
  - /home/ercel001/AUVIS/task5D_ml_prototype/data/embeddings/train_embeddings.npz

Gefundene Metadaten (.parquet):
  - /home/ercel001/AUVIS/task5D_ml_prototype/data/prepared/dev_utterances.parquet
  - /home/ercel001/AUVIS/task5D_ml_prototype/data/prepared/dev_utterances_multisession.parquet
  - /home/ercel001/AUVIS/task5D_ml_prototype/data/prepared/train_utterances.parquet
  - /home/ercel001/AUVIS/task5D_ml_prototype/data/prepared/train_utterances_multisession.parquet

Gefundene speaker_to_cluster.json:
  - /home/ercel001/AUVIS/task5D_ml_prototype/data/raw/dev/dev/session_132/labels/speaker_to_cluster.json
  - /home/ercel001/AUVIS/task5D_ml_prototype/data/raw/dev/dev/session_133/labels/speaker_to_cluster.json
  - /home/ercel001/AUVIS/task5D_ml_prototype/data/raw/dev/dev/session_134/labels/speaker_to_cluster.json
  - /home/ercel001/AUVIS/task5D_ml_prototype/data/raw/dev/dev/sessi

## 3. Manuelle Konfiguration (optional)

Standardmäßig werden die zuletzt gefundenen Dateien aus der Auto-Discovery genutzt.  
Falls diese nicht passen, können hier die Pfade sowie zentrale Parameter manuell gesetzt werden.  
(Dieser Abschnitt kann bei funktionierender Auto-Discovery übersprungen oder auskommentiert werden.)


In [3]:
# ============================================================
# 3) Manuelle Konfiguration (optional)
# ============================================================

# Standardfall: Auto-Discovery-Defaults verwenden.
# → Wenn du manuell überschreiben willst, setze einfach unten einen eigenen Pfad.
EMB_NPZ_FILE = DEFAULT_EMB_NPZ            # z.B. Path("/…/train_embeddings.npz")
META_PARQUET_FILE = DEFAULT_META_PARQUET  # z.B. Path("/…/train_utterances.parquet")
SPEAKER_TO_CLUSTER_JSON = DEFAULT_STC_JSON  # optional; nur nötig, falls im Parquet kein Cluster steht

# Parameter für die Paarbildung / Modelle (bei Bedarf anpassen)
MAX_POS_PAIRS_PER_UTT = 2
NEGATIVE_MULTIPLIER = 2
MAX_TOTAL_PAIRS = 300_000
TIME_WINDOW_S = None

USE_SVM = False
N_ESTIMATORS_RF = 300
N_SPLITS_GROUPKFOLD = 5
TEST_SIZE = 0.2

print("Konfiguration (aktiv):")
print("  EMB_NPZ_FILE=", EMB_NPZ_FILE)
print("  META_PARQUET_FILE=", META_PARQUET_FILE)
print("  SPEAKER_TO_CLUSTER_JSON=", SPEAKER_TO_CLUSTER_JSON)
print("  MAX_POS_PAIRS_PER_UTT=", MAX_POS_PAIRS_PER_UTT)
print("  NEGATIVE_MULTIPLIER=", NEGATIVE_MULTIPLIER)
print("  MAX_TOTAL_PAIRS=", MAX_TOTAL_PAIRS)
print("  TIME_WINDOW_S=", TIME_WINDOW_S)
print("  USE_SVM=", USE_SVM)
print("  N_ESTIMATORS_RF=", N_ESTIMATORS_RF)
print("  N_SPLITS_GROUPKFOLD=", N_SPLITS_GROUPKFOLD)
print("  TEST_SIZE=", TEST_SIZE)


Konfiguration (aktiv):
  EMB_NPZ_FILE= /home/ercel001/AUVIS/task5D_ml_prototype/data/embeddings/dev_embeddings.npz
  META_PARQUET_FILE= /home/ercel001/AUVIS/task5D_ml_prototype/data/prepared/dev_utterances_multisession.parquet
  SPEAKER_TO_CLUSTER_JSON= /home/ercel001/AUVIS/task5D_ml_prototype/data/raw/dev/dev/session_137/labels/speaker_to_cluster.json
  MAX_POS_PAIRS_PER_UTT= 2
  NEGATIVE_MULTIPLIER= 2
  MAX_TOTAL_PAIRS= 300000
  TIME_WINDOW_S= None
  USE_SVM= False
  N_ESTIMATORS_RF= 300
  N_SPLITS_GROUPKFOLD= 5
  TEST_SIZE= 0.2


## 4. Daten laden & prüfen

In diesem Abschnitt werden die **Embeddings** (`.npz`) und die **Metadaten** (`.parquet`) geladen.  
Erwartet werden mindestens folgende Felder (ggf. unter abweichenden Spaltennamen, die wir robust abfangen):

- `utt_id` (eindeutige ID einer Äußerung)  
- `speaker_id`  
- `session_id`  
- Zeitfelder: `start`, `end` (oder Varianten)  
- `text`  
- **Konversations-Label**: z. B. `cluster`, `conv_id` (falls nicht vorhanden, wird – wenn verfügbar – `speaker_to_cluster.json` gejoint)


In [4]:
# ============================================================
# 4) Daten laden & prüfen
# ============================================================

def load_embeddings(npz_path: Path):
    """
    Lädt Embeddings (.npz) und gibt (utt_ids, embeddings) zurück.
    Erwartet:
      - Schlüssel mit 'utt' oder 'id' → Äußerungs-IDs
      - Schlüssel mit 'emb'          → Embedding-Matrix
    """
    if npz_path is None or not Path(npz_path).exists():
        raise FileNotFoundError(f"Embeddings-Datei nicht gefunden: {npz_path}")
    data = np.load(npz_path, allow_pickle=True)

    id_keys = [k for k in data.files if "utt" in k or "id" in k]
    emb_keys = [k for k in data.files if "emb" in k]

    if not id_keys or not emb_keys:
        raise KeyError(f"Unerwartete Schlüssel in {npz_path}. Gefunden: {data.files}")

    utt_ids = data[id_keys[0]]
    X = data[emb_keys[0]]

    assert len(utt_ids) == len(X), "Länge von utt_ids und embeddings passt nicht"
    return utt_ids.astype(str), X.astype(np.float32)


def parse_time_to_seconds(x):
    """Konvertiert verschiedene Zeitformate in Sekunden."""
    if pd.isna(x):
        return np.nan
    if isinstance(x, (int, float, np.integer, np.floating)):
        return float(x)

    s = str(x)
    try:
        td = pd.to_timedelta(s)
        return td.total_seconds()
    except Exception:
        pass
    try:
        parts = s.split(':')
        if len(parts) == 3:
            h, m, sec = parts
            return int(h)*3600 + int(m)*60 + float(sec)
    except Exception:
        return np.nan
    return np.nan


def robust_column(df: pd.DataFrame, candidates, required: bool = True):
    """
    Gibt den ersten Spaltennamen zurück, der im DataFrame vorhanden ist.
    Falls required=True und keine Treffer → KeyError.
    """
    for c in candidates:
        if c in df.columns:
            return c
    if required:
        raise KeyError(f"Erforderliche Spalte nicht gefunden. Kandidaten: {candidates}")
    return None


# --- Metadaten laden ---
if META_PARQUET_FILE is None or not Path(META_PARQUET_FILE).exists():
    raise FileNotFoundError(f"Metadaten-Parquet nicht gefunden: {META_PARQUET_FILE}")

df_meta = pd.read_parquet(META_PARQUET_FILE)
print("Metadaten geladen:", df_meta.shape)

# Spalten robust identifizieren
utt_col     = robust_column(df_meta, ["utt_id", "utt", "utterance_id", "id"])
speaker_col = robust_column(df_meta, ["speaker_id", "spk", "speaker"])
session_col = robust_column(df_meta, ["session_id", "session", "sess_id"])
start_col   = robust_column(df_meta, ["start", "start_time", "ts_start", "begin"], required=False)
end_col     = robust_column(df_meta, ["end", "end_time", "ts_end", "stop"], required=False)
text_col    = robust_column(df_meta, ["text", "transcript", "utt_text"], required=False)

# Cluster-Spalte bestimmen oder speaker_to_cluster.json joinen
cluster_col = None
for cand in ["cluster", "conv_id", "conversation_id", "cluster_id", "conv"]:
    if cand in df_meta.columns:
        cluster_col = cand
        break

if cluster_col is None and SPEAKER_TO_CLUSTER_JSON and Path(SPEAKER_TO_CLUSTER_JSON).exists():
    with open(SPEAKER_TO_CLUSTER_JSON, "r", encoding="utf-8") as f:
        stc = json.load(f)
    rows = []
    if isinstance(stc, dict):
        for sess_k, mapping in stc.items():
            if isinstance(mapping, dict):
                for spk_k, cl in mapping.items():
                    rows.append({"session_id": str(sess_k), "speaker_id": str(spk_k), "cluster": cl})
    stc_df = pd.DataFrame(rows)
    if not stc_df.empty:
        stc_df["session_id"] = stc_df["session_id"].astype(str)
        stc_df["speaker_id"] = stc_df["speaker_id"].astype(str)
        df_meta[session_col] = df_meta[session_col].astype(str)
        df_meta[speaker_col] = df_meta[speaker_col].astype(str)
        df_meta = df_meta.merge(stc_df, left_on=[session_col, speaker_col],
                                right_on=["session_id", "speaker_id"], how="left")
        cluster_col = "cluster"

if cluster_col is None:
    raise KeyError("Keine Konversations-/Cluster-Spalte gefunden. "
                   "Bitte sicherstellen, dass Teil 2/4 die Cluster im Parquet enthalten "
                   "oder 'SPEAKER_TO_CLUSTER_JSON' angeben.")

# Zeitspalten konvertieren
if start_col:
    df_meta["start_s"] = df_meta[start_col].map(parse_time_to_seconds)
if end_col:
    df_meta["end_s"] = df_meta[end_col].map(parse_time_to_seconds)

# Textlänge
if text_col:
    df_meta["text_len"] = df_meta[text_col].astype(str).map(lambda s: len(s.split()))

# --- Embeddings laden ---
if EMB_NPZ_FILE is None or not Path(EMB_NPZ_FILE).exists():
    raise FileNotFoundError(f"Embeddings-Datei nicht gefunden: {EMB_NPZ_FILE}")

utt_ids, X = load_embeddings(Path(EMB_NPZ_FILE))
print("Embeddings geladen:", X.shape)

# Schnittmenge bilden
before = len(df_meta)
utt_set = set(utt_ids.tolist())
df_meta = df_meta[df_meta[utt_col].astype(str).isin(utt_set)].copy()
after = len(df_meta)

print(f"Gefiltert: {before} → {after} (nach Schnittmenge mit Embeddings)")

# Sortieren nach Session + Zeit
df_meta = df_meta.sort_values([session_col, "start_s" if "start_s" in df_meta.columns else utt_col]).reset_index(drop=True)

print("Finale Metadaten:", df_meta.shape)
print("Vorhandene Spalten (erste 30):", sorted(df_meta.columns.tolist())[:30])


Metadaten geladen: (8561, 7)
Embeddings geladen: (8561, 768)
Gefiltert: 8561 → 8561 (nach Schnittmenge mit Embeddings)
Finale Metadaten: (8561, 8)
Vorhandene Spalten (erste 30): ['cluster', 'end_s', 'session_id', 'speaker_id', 'start_s', 'text', 'text_len', 'utt_id']


## 5. Konstruktion des Paar-Datasets

Für jede **Session** werden Paare `(i, j)` von Äußerungen gebildet:  

- **Positive Paare:** beide Äußerungen im gleichen Konversations-Cluster  
- **Negative Paare:** Äußerungen aus unterschiedlichen Clustern derselben Session  

Um Rechenaufwand zu kontrollieren, nutzen wir Sampling-Parameter:  
- `MAX_POS_PAIRS_PER_UTT`: max. Anzahl positiver Paare pro Äußerung (zufällig gewählt)  
- `NEGATIVE_MULTIPLIER`: Faktor für negative Paare je positivem Paar  
- `TIME_WINDOW_S`: optionaler Zeitfilter (|Δt| ≤ Zeitfenster)

**Features pro Paar:**  
- `cos_sim`, `time_gap_s` (+ `log_time_gap`), `overlap`, `same_speaker`  
- Dauer (`dur_i`, `dur_j`, `duration_diff`)  
- Textlängen (`len_i`, `len_j`, `len_diff`)  
- `turn_distance` (Unterschied der Position im Gespräch)


In [5]:
# ============================================================
# 5) Konstruktion des Paar-Datasets
# ============================================================

# Mapping: Utterance-ID -> Index im Embedding-Array
utt_to_idx = {str(u): i for i, u in enumerate(utt_ids)}

def l2_normalize_rows(M: np.ndarray) -> np.ndarray:
    """
    Normiert jede Zeile einer Matrix auf Länge 1 (L2-Norm).
    Verhindert, dass Cosine Similarity von der Betragsgröße beeinflusst wird.
    """
    norms = np.linalg.norm(M, axis=1, keepdims=True) + 1e-9
    return M / norms

# Vorberechnete normalisierte Embeddings
X_norm = l2_normalize_rows(X)

def cos_sim_by_ids(u1: str, u2: str) -> float:
    """
    Berechnet die Cosine Similarity zwischen zwei Utterances anhand ihrer IDs.
    Gibt NaN zurück, falls eine ID fehlt.
    """
    i1 = utt_to_idx.get(str(u1), None)
    i2 = utt_to_idx.get(str(u2), None)
    if i1 is None or i2 is None:
        return np.nan
    return float(np.dot(X_norm[i1], X_norm[i2]))

def make_pairs_for_session(df_sess: pd.DataFrame) -> pd.DataFrame:
    """
    Erzeugt Paare (positive & negative) für eine Session und extrahiert Features.

    Schritte:
      1. Positive Paare: innerhalb desselben Clusters
      2. Negative Paare: zufällig zwischen Clustern (gesteuert über NEGATIVE_MULTIPLIER)
      3. Feature-Berechnung für jedes Paar (Ähnlichkeiten, Zeitabstände, etc.)
    """
    rows = []

    # Session sortieren nach Startzeit oder Index
    order = df_sess.sort_values("start_s" if "start_s" in df_sess.columns else df_sess.index).reset_index(drop=True)
    order["turn_idx"] = np.arange(len(order))  # Reihenfolge im Gespräch

    # --------------------------------------------------------
    # 1) Positive Paare
    # --------------------------------------------------------
    for clust, df_c in order.groupby(cluster_col):
        ids_c = df_c[utt_col].astype(str).tolist()
        if len(ids_c) < 2:
            continue
        for u in ids_c:
            partners = [v for v in ids_c if v != u]
            if not partners:
                continue
            np.random.shuffle(partners)
            partners = partners[:MAX_POS_PAIRS_PER_UTT]
            for v in partners:
                rows.append((u, v, 1))  # Label = 1 (gleiches Gespräch)

    pos_count = sum(1 for _, _, y in rows if y == 1)
    neg_target = pos_count * NEGATIVE_MULTIPLIER

    # --------------------------------------------------------
    # 2) Negative Paare
    # --------------------------------------------------------
    df_by_cluster = {c: d for c, d in order.groupby(cluster_col)}
    clusters = list(df_by_cluster.keys())
    utts_by_cluster = {c: d[utt_col].astype(str).tolist() for c, d in df_by_cluster.items()}

    # Hilfstabellen für schnelle Zugriffe
    start_map = dict(zip(order[utt_col].astype(str), order["start_s" if "start_s" in order.columns else utt_col]))
    end_map   = dict(zip(order[utt_col].astype(str), order["end_s" if "end_s" in order.columns else utt_col]))
    spk_map   = dict(zip(order[utt_col].astype(str), order[speaker_col].astype(str)))
    turn_map  = dict(zip(order[utt_col].astype(str), order["turn_idx"]))
    len_map   = dict(zip(order[utt_col].astype(str), order.get("text_len", pd.Series([np.nan]*len(order))).fillna(np.nan)))

    dur_map = {}
    for u in order[utt_col].astype(str):
        s = start_map.get(u, np.nan)
        e = end_map.get(u, np.nan)
        dur_map[u] = (e - s) if (isinstance(s, float) and isinstance(e, float)) else np.nan

    neg_rows = []
    attempts = 0
    max_attempts = neg_target * 10 + 1000  # Sicherheitslimit
    while len(neg_rows) < neg_target and attempts < max_attempts:
        attempts += 1
        if len(clusters) < 2:
            break
        c1, c2 = np.random.choice(clusters, size=2, replace=False)
        u = np.random.choice(utts_by_cluster[c1])
        v = np.random.choice(utts_by_cluster[c2])
        if TIME_WINDOW_S is not None:
            t1 = start_map.get(u, np.nan)
            t2 = start_map.get(v, np.nan)
            if (isinstance(t1, float) and isinstance(t2, float)) and abs(t1 - t2) > TIME_WINDOW_S:
                continue
        neg_rows.append((u, v, 0))  # Label = 0 (verschiedene Gespräche)

    rows.extend(neg_rows)

    # --------------------------------------------------------
    # 3) Feature-Berechnung
    # --------------------------------------------------------
    feats = []
    for u, v, y in rows:
        cs = cos_sim_by_ids(u, v)
        t1, e1 = start_map.get(u, np.nan), end_map.get(u, np.nan)
        t2, e2 = start_map.get(v, np.nan), end_map.get(v, np.nan)

        time_gap = np.nan
        overlap = 0
        dur1 = np.nan
        dur2 = np.nan
        if isinstance(t1, float) and isinstance(t2, float):
            time_gap = abs(t2 - t1)
        if isinstance(t1, float) and isinstance(e1, float) and isinstance(t2, float) and isinstance(e2, float):
            overlap = int((e1 > t2) and (e2 > t1))
            dur1 = e1 - t1
            dur2 = e2 - t2

        same_spk = int(spk_map.get(u) == spk_map.get(v))

        turn_diff = np.nan
        if u in turn_map and v in turn_map:
            turn_diff = abs(int(turn_map[u]) - int(turn_map[v]))

        len1 = len_map.get(u, np.nan)
        len2 = len_map.get(v, np.nan)

        feats.append({
            "u": u, "v": v, "label": y,
            "cos_sim": cs,
            "time_gap_s": time_gap,
            "log_time_gap": math.log1p(time_gap) if isinstance(time_gap, float) else np.nan,
            "overlap": overlap,
            "same_speaker": same_spk,
            "duration_i": dur1, "duration_j": dur2,
            "duration_diff": (dur1 - dur2) if (isinstance(dur1, float) and isinstance(dur2, float)) else np.nan,
            "len_i": len1, "len_j": len2,
            "len_diff": (len1 - len2) if (isinstance(len1, (int, float)) and isinstance(len2, (int, float))) else np.nan,
            "turn_distance": turn_diff,
        })
    return pd.DataFrame(feats)

# ------------------------------------------------------------
# Paarbildung über alle Sessions
# ------------------------------------------------------------
pairs_list = []
total_pairs = 0
for sess, df_sess in tqdm(df_meta.groupby(session_col), desc="Sessions"):
    df_pairs = make_pairs_for_session(df_sess)
    if df_pairs.empty:
        continue
    pairs_list.append(df_pairs)
    total_pairs += len(df_pairs)
    if total_pairs >= MAX_TOTAL_PAIRS:
        break

pairs = pd.concat(pairs_list, ignore_index=True) if pairs_list else pd.DataFrame(columns=[
    "u","v","label","cos_sim","time_gap_s","log_time_gap","overlap","same_speaker",
    "duration_i","duration_j","duration_diff","len_i","len_j","len_diff","turn_distance"
])

print("Paar-Dataset:", pairs.shape)
display(pairs.sample(min(5, len(pairs))) if len(pairs) else pairs)

# Speicherung des Datasets
out_pairs = PROJECT_ROOT / "data" / "derived" / "part5_pairs" / "pairs_dataset.parquet"
out_pairs.parent.mkdir(parents=True, exist_ok=True)
pairs.to_parquet(out_pairs, index=False)
print("Gespeichert:", out_pairs)


Sessions:   0%|          | 0/25 [00:00<?, ?it/s]

Paar-Dataset: (17122, 15)


,u,v,label,cos_sim,time_gap_s,log_time_gap,overlap,same_speaker,duration_i,duration_j,duration_diff,len_i,len_j,len_diff,turn_distance
12933,session_51_4_0022,session_51_4_0055,1,0.075796,NaN,NaN,0,1,NaN,NaN,NaN,1,3,-2,193
15843,session_55_2_0047,session_55_1_0030,1,0.021799,NaN,NaN,0,0,NaN,NaN,NaN,10,5,5,126
11364,session_49_0_0005,session_49_0_0009,1,0.111037,NaN,NaN,0,1,NaN,NaN,NaN,8,6,2,59
6911,session_141_3_0051,session_141_1_0047,1,0.165890,NaN,NaN,0,0,NaN,NaN,NaN,2,1,1,31
15842,session_55_2_0047,session_55_1_0019,1,-0.061049,NaN,NaN,0,0,NaN,NaN,NaN,10,11,-1,176


Gespeichert: /home/ercel001/AUVIS/task5D_ml_prototype/data/derived/part5_pairs/pairs_dataset.parquet


## 6. Train/Test-Splits (gruppenbasiert) & Modellierung

Um **Leckagen** zu vermeiden, wird der Split **gruppenbasiert nach Session** durchgeführt.  
Dadurch können keine Utterances derselben Session gleichzeitig im Training und Testset landen.  

Wir trainieren:  
- **Logistische Regression** (mit Standardisierung)  
- **Random Forest** (ohne Standardisierung notwendig)  
- optional **SVM** (mit Standardisierung)  

Zusätzlich erfolgt eine **gruppenbasierte Cross-Validation (GroupKFold)** auf dem Trainingsset.


In [6]:
# ============================================================
# 6) Train/Test-Splits (gruppenbasiert) & Modellierung
# ============================================================

from sklearn.model_selection import GroupShuffleSplit
import numpy as np

# ------------------------------------------------------------
# Mapping von Utterance-ID -> Session-ID
# ------------------------------------------------------------
utt2sess = dict(zip(df_meta[utt_col].astype(str), df_meta[session_col].astype(str)))

# Session-Zuordnung für beide Utterances im Paar
pairs["session_u"] = pairs["u"].map(utt2sess)
pairs["session_v"] = pairs["v"].map(utt2sess)

# Nur Paare behalten, die aus derselben Session stammen
pairs = pairs[pairs["session_u"] == pairs["session_v"]].copy()
pairs["session"] = pairs["session_u"]

# ------------------------------------------------------------
# Feature-Auswahl
# ------------------------------------------------------------
feature_cols = [
    "cos_sim", "time_gap_s", "log_time_gap", "overlap", "same_speaker",
    "duration_i", "duration_j", "duration_diff",
    "len_i", "len_j", "len_diff", "turn_distance"
]

X_df = pairs[feature_cols].copy()
y = pairs["label"].astype(int).values
groups = pairs["session"].astype(str).values  # Sessions als Gruppen für Splits

# ------------------------------------------------------------
# Vorverarbeitung: NaN & Inf behandeln
# ------------------------------------------------------------
X_df = X_df.replace([np.inf, -np.inf], np.nan)
medians = X_df.median(numeric_only=True)  # Median je Spalte
X_df = X_df.fillna(medians)               # fehlende Werte ersetzen
Xn = X_df.values                          # numpy-Array für Modelle

# ------------------------------------------------------------
# Gruppenbasierter Split in Train/Test
# ------------------------------------------------------------
try:
    TEST_SIZE
except NameError:
    TEST_SIZE = 0.2  # Default, falls nicht gesetzt

gss = GroupShuffleSplit(n_splits=1, test_size=TEST_SIZE, random_state=42)
train_idx, test_idx = next(gss.split(Xn, y, groups))

X_train, X_test = Xn[train_idx], Xn[test_idx]
y_train, y_test = y[train_idx], y[test_idx]
groups_train = groups[train_idx]

print(f"Train: {X_train.shape}, Test: {X_test.shape}")


Train: (13822, 12), Test: (3300, 12)


## 7) Modelle definieren & trainieren

Wir verwenden je ein **Pipeline‑Objekt** pro Modell:

- `LogisticRegression` (mit `class_weight='balanced'`)
- `RandomForestClassifier` (mit `class_weight='balanced_subsample'`)
- Optional: `SVC` (RBF‑Kernel, `probability=True`, `class_weight='balanced'`) – **vorsichtig** bei großen Datenmengen


In [7]:
# ============================================================
# 7) Modelle definieren & trainieren (mit Class-Balance-Check)
# ============================================================

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
import numpy as np

# Defaults
N_ESTIMATORS_RF = globals().get("N_ESTIMATORS_RF", 200)
USE_SVM = globals().get("USE_SVM", False)

# ------------------------------------------------------------
# Helper: check label balance
# ------------------------------------------------------------
def has_two_classes(y):
    return len(np.unique(y)) > 1

print("Train-Klassenverteilung:", np.bincount(y_train))
print("Test-Klassenverteilung: ", np.bincount(y_test))

if not has_two_classes(y_train):
    print("[WARN] Trainset enthält nur eine Klasse – Split neu ziehen!")
    from sklearn.model_selection import GroupShuffleSplit
    gss = GroupShuffleSplit(n_splits=5, test_size=TEST_SIZE, random_state=42)
    for train_idx, test_idx in gss.split(Xn, y, groups):
        y_tmp = y[train_idx]
        if has_two_classes(y_tmp):
            X_train, X_test = Xn[train_idx], Xn[test_idx]
            y_train, y_test = y_tmp, y[test_idx]
            groups_train = groups[train_idx]
            break
    print("Neue Verteilung:", np.bincount(y_train))

# ------------------------------------------------------------
# Pipelines
# ------------------------------------------------------------
imputer = SimpleImputer(strategy="median")

logreg = Pipeline([
    ("imputer", imputer),
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(
        max_iter=1000,
        class_weight="balanced",
        random_state=42
    ))
])

rf = Pipeline([
    ("imputer", imputer),
    ("clf", RandomForestClassifier(
        n_estimators=N_ESTIMATORS_RF,
        n_jobs=-1,
        class_weight="balanced_subsample",
        random_state=42
    ))
])

svm = None
if USE_SVM:
    svm = Pipeline([
        ("imputer", imputer),
        ("scaler", StandardScaler()),
        ("clf", SVC(
            kernel="rbf",
            probability=True,
            class_weight="balanced",
            random_state=42
        ))
    ])

models = {"LogReg": logreg, "RandomForest": rf}
if svm is not None:
    models["SVM"] = svm

# ------------------------------------------------------------
# Training
# ------------------------------------------------------------
for name, model in models.items():
    print(f"\n=== Trainiere {name} ===")
    model.fit(X_train, y_train)

print("\nFertig.")


Train-Klassenverteilung: [    0 13822]
Test-Klassenverteilung:  [   0 3300]
[WARN] Trainset enthält nur eine Klasse – Split neu ziehen!
Neue Verteilung: [    0 13822]

=== Trainiere LogReg ===


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1

In [9]:
print("NEGATIVE_MULTIPLIER =", NEGATIVE_MULTIPLIER)
print("TIME_WINDOW_S =", TIME_WINDOW_S)

# Cluster-Anzahl je Session
cluster_counts = df_meta.groupby(session_col)[cluster_col].nunique()
print("\nCluster-Anzahl je Session:")
print(cluster_counts.describe())
print(cluster_counts.value_counts().head(10))


NEGATIVE_MULTIPLIER = 2
TIME_WINDOW_S = None

Cluster-Anzahl je Session:
count    25.0
mean      1.0
std       0.0
min       1.0
25%       1.0
50%       1.0
75%       1.0
max       1.0
Name: cluster, dtype: float64
cluster
1    25
Name: count, dtype: int64


## 5) Evaluation (Testset) & Cross‑Validation

Wir berichten **Accuracy, Precision, Recall, F1, ROC‑AUC** und zeigen eine **Konfusionsmatrix**.  
Zusätzlich führen wir eine **gruppenbasierte CV** (GroupKFold) auf dem Trainingsset durch, um die Stabilität zu prüfen.


In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support,
    roc_auc_score, classification_report, confusion_matrix, roc_curve
)
from sklearn.model_selection import GroupKFold
import numpy as np
import matplotlib.pyplot as plt

def evaluate_model(name: str, model, X_tr, y_tr, X_te, y_te):
    y_pred = model.predict(X_te)
    try:
        y_proba = model.predict_proba(X_te)[:, 1]
    except Exception:
        y_proba = None
    
    acc = accuracy_score(y_te, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y_te, y_pred, average="binary", zero_division=0)
    auc = roc_auc_score(y_te, y_proba) if y_proba is not None else np.nan
    
    print(f"\n[{name}] Test-Ergebnisse:")
    print(f"  Accuracy : {acc:.4f}")
    print(f"  Precision: {prec:.4f}")
    print(f"  Recall   : {rec:.4f}")
    print(f"  F1       : {f1:.4f}")
    print(f"  ROC-AUC  : {auc:.4f}")
    print("\nClassification Report:\n", classification_report(y_te, y_pred, digits=3))
    
    cm = confusion_matrix(y_te, y_pred, labels=[0,1])
    print("Konfusionsmatrix (Zeile=Ist, Spalte=Vorhersage):\n", cm)
    
    if y_proba is not None:
        fpr, tpr, _ = roc_curve(y_te, y_proba)
        plt.figure(figsize=(5,4))
        plt.plot(fpr, tpr, label=f"{name} (AUC={auc:.3f})")
        plt.plot([0,1],[0,1], linestyle='--')
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title(f"ROC-Kurve – {name}")
        plt.legend(loc="lower right")
        plt.show()

for name, model in models.items():
    evaluate_model(name, model, X_train, y_train, X_test, y_test)

print("\nGruppenbasierte Cross-Validation (nur Trainingsteil):")
gkf = GroupKFold(n_splits=N_SPLITS_GROUPKFOLD)
for name, model in models.items():
    scores = []
    for fold, (tr_idx, va_idx) in enumerate(gkf.split(X_train, y_train, groups=groups_train), 1):
        mdl = model
        mdl.fit(X_train[tr_idx], y_train[tr_idx])
        y_pred = mdl.predict(X_train[va_idx])
        try:
            y_proba = mdl.predict_proba(X_train[va_idx])[:, 1]
            auc = roc_auc_score(y_train[va_idx], y_proba)
        except Exception:
            auc = np.nan
        f1 = precision_recall_fscore_support(y_train[va_idx], y_pred, average="binary", zero_division=0)[2]
        scores.append((f1, auc))
    f1_mean = float(np.nanmean([s[0] for s in scores]))
    auc_mean = float(np.nanmean([s[1] for s in scores]))
    print(f"  {name}: mean F1={f1_mean:.4f}, mean ROC-AUC={auc_mean:.4f} (über {N_SPLITS_GROUPKFOLD} Folds)")

## 6) Feature‑Bedeutung & Fehleranalyse

Wir untersuchen, welche Merkmale besonders relevant sind (LR‑Koeffizienten, RF‑Feature‑Importances) und betrachten **Fehlklassifikationen** (Top‑FP/FN) stichprobenartig.


In [ ]:
import numpy as np
import pandas as pd

def show_feature_importance(model_name: str, model, feature_names):
    try:
        if hasattr(model, "named_steps"):
            final = model.named_steps.get("clf", model)
        else:
            final = model
        if hasattr(final, "coef_"):
            coef = np.ravel(final.coef_)
            imp_df = pd.DataFrame({"feature": feature_names, "importance": coef})
            imp_df = imp_df.sort_values("importance", key=lambda s: s.abs(), ascending=False)
            print(f"\nLR-Koeffizienten ({model_name}):")
            display(imp_df)
        elif hasattr(final, "feature_importances_"):
            fi = final.feature_importances_
            imp_df = pd.DataFrame({"feature": feature_names, "importance": fi})
            imp_df = imp_df.sort_values("importance", ascending=False)
            print(f"\nRF-Feature Importances ({model_name}):")
            display(imp_df)
    except Exception as e:
        print(f"Konnte Feature-Bedeutungen für {model_name} nicht darstellen: {e}")

for name, model in models.items():
    show_feature_importance(name, model, feature_cols)

def error_samples(model, X_te, y_te, df_pairs_test: pd.DataFrame, n=10):
    y_pred = model.predict(X_te)
    try:
        y_proba = model.predict_proba(X_te)[:,1]
    except Exception:
        y_proba = np.full_like(y_pred, fill_value=np.nan, dtype=float)
    df = df_pairs_test.copy()
    df["y_true"] = y_te
    df["y_pred"] = y_pred
    df["score"] = y_proba
    fp = df[(df.y_true==0) & (df.y_pred==1)].sort_values("score", ascending=False).head(n)
    fn = df[(df.y_true==1) & (df.y_pred==0)].sort_values("score", ascending=True).head(n)
    return fp, fn

pairs_test = pairs.iloc[test_idx].copy()
for name, model in models.items():
    print(f"\nFehleranalyse – {name}")
    fp, fn = error_samples(model, X_test, y_test, pairs_test, n=10)
    print("Top False Positives:")
    display(fp[["u","v","label","score","cos_sim","time_gap_s","same_speaker","turn_distance"]])
    print("Top False Negatives:")
    display(fn[["u","v","label","score","cos_sim","time_gap_s","same_speaker","turn_distance"]])

## 7) Interpretation & Integration in die Pipeline

**Interpretation:** Ein ML‑basiertes Paar‑Modell kann **schwierige Fälle** besser unterscheiden, da es mehrere Signale kombiniert (Semantik + Zeit + Struktur).  
In ersten Experimenten sind insbesondere `cos_sim` und `time_gap_s` oft starke Prädiktoren; `overlap` und `same_speaker` liefern zusätzlichen Kontext.

**Integration in Task‑5D Pipeline (Vorschlag):**
1. Erzeuge für jede Session einen **Graphen**: Knoten = Utterances, Kante `(i, j)` wenn `P(same_conversation) ≥ τ` und ggf. `|Δt| ≤ W` (Heuristik).  
2. Bestimme **verbundene Komponenten** oder wende **community detection** / **graph clustering** an, um Gesprächsgruppen zu erhalten.  
3. **Kalibriere** den Schwellwert `τ` per Dev‑Set auf **conversation‑level** Metriken (z. B. ARI, B‑Cubed F1), nicht nur pair‑level F1.  
4. Kombiniere mit Distanz‑ oder Overlap‑Heuristiken aus Teil 4 für **Hybrid‑Ansätze** (z. B. nur Kandidaten‑Kanten innerhalb eines Zeitfensters).

**Bewertung:** Ob der Ansatz die Gesprächstrennung **robuster** macht, zeigt sich daran, ob die resultierenden Konversationscluster auf Session‑Ebene stabiler und näher an den Referenzlabels liegen als beim rein distanzbasierten Verfahren. Für eine faire Bewertung sollte dieselbe Train/Test‑Aufteilung je Session verwendet werden.


## 8) Persistenz & Nächste Schritte

- Das Paar‑Dataset wurde unter `data/derived/part5_pairs/pairs_dataset.parquet` abgelegt.  
- Modelle lassen sich optional mit `joblib` speichern und später wiederverwenden.  
- **Nächste Schritte:**
  - Hyperparameter‑Suche (z. B. `class_weight`, `C`, `max_depth`, `threshold τ`)
  - Threshold‑Kalibrierung gegen konversationsbasierte Metriken (ARI, B‑Cubed)
  - Erweiterte Merkmale (z. B. Sprecherwechsel‑Dichte, semantische Themenwechsel, Prosodie‑Features)
  - Graph‑basierte Post‑Processing‑Strategien


In [ ]:
# Optional: Modelle speichern (bei Bedarf aktivieren)
# from joblib import dump
# from pathlib import Path
# def ensure_dir(p: Path):
#     p.mkdir(parents=True, exist_ok=True)
# ensure_dir(Path("data/derived/part5_pairs/models"))
# for name, model in models.items():
#     outp = Path("data/derived/part5_pairs/models") / f"{name}_pair_classifier.joblib"
#     dump(model, outp)
#     print("Gespeichert:", outp)